In [61]:
import numpy as np
import pandas as pd
import math
import timeit
import time
import string
from matplotlib import pyplot

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Energy41.csv to Energy41 (1).csv


In [62]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Energy41.csv']))
dataset.head(40)

,dateRep,Outflow HHP1,Inflow HPP1,Inflow HPP2,Production HPP1,Production HPP2
0,2006/01/01,"158,87155322267287",257,55,2870,4350
1,2006/01/02,"168,01416144057697",253,59,1960,4000
2,2006/01/03,"113,97097279628683",327,67,1886,3740
3,2006/01/04,"108,87795296245295",309,84,1280,3240
4,2006/01/05,"73,08660464134628",262,79,2400,5020
5,2006/01/06,"135,69053343967414",248,84,2570,3440
6,2006/01/07,"145,51706462474957",230,76,2450,4430
7,2006/01/08,"138,3687397283832",186,65,2290,4160
8,2006/01/09,"128,5916763558813",238,62,2290,4000
9,2006/01/10,"128,06411185173383",227,57,2150,3640


In [63]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2831 entries, 0 to 2830
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   dateRep          2831 non-null   object
 1   Outflow HHP1     2823 non-null   object
 2   Inflow  HPP1     2830 non-null   object
 3   Inflow HPP2      2830 non-null   object
 4   Production HPP1  2821 non-null   object
 5   Production HPP2  2830 non-null   object
dtypes: object(6)
memory usage: 132.8+ KB


In [64]:
perc =[.20, .40, .60, .80] 
include =['object', 'float', 'int'] 
dataset.describe(percentiles = perc, include = include)

,dateRep,Outflow HHP1,Inflow HPP1,Inflow HPP2,Production HPP1,Production HPP2
count,2831,2823,2830,2830,2821,2830
unique,2831,2530,2658,2526,1996,1998
top,2008/09/13,0,40,15,0,4600
freq,1,250,18,30,251,10


In [65]:
dataset.dtypes

dateRep            object
Outflow HHP1       object
Inflow  HPP1       object
Inflow HPP2        object
Production HPP1    object
Production HPP2    object
dtype: object

In [66]:
for i in range(1,len(dataset.columns)):
  name = dataset.columns[i]
  dataset[name] = dataset[name].str.replace(',','.').astype(float)

In [ ]:
dataset['Outflow HHP1'] = dataset['Outflow HHP1'].astype(np.float64)
dataset['Inflow  HPP1'] = dataset['Inflow  HPP1'].astype(np.float64)
dataset['Inflow HPP2'] = dataset['Inflow HPP2'].astype(np.float64)
dataset['Production HPP1'] = dataset['Production HPP1'].astype(np.float64)
dataset['Production HPP2'] = dataset['Production HPP2'].astype(np.float64)

In [67]:
dataset.head(5)

,dateRep,Outflow HHP1,Inflow HPP1,Inflow HPP2,Production HPP1,Production HPP2
0,2006/01/01,158.871553,257.0,55.0,2870.0,4350.0
1,2006/01/02,168.014161,253.0,59.0,1960.0,4000.0
2,2006/01/03,113.970973,327.0,67.0,1886.0,3740.0
3,2006/01/04,108.877953,309.0,84.0,1280.0,3240.0
4,2006/01/05,73.086605,262.0,79.0,2400.0,5020.0


In [68]:
dataset.dtypes

dateRep             object
Outflow HHP1       float64
Inflow  HPP1       float64
Inflow HPP2        float64
Production HPP1    float64
Production HPP2    float64
dtype: object

In [69]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [70]:
missing_values_table(dataset)

Your selected dataframe has 6 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
Production HPP1,10,0.4
Outflow HHP1,8,0.3
Inflow HPP1,1,0.0
Inflow HPP2,1,0.0
Production HPP2,1,0.0


In [71]:
print("Replacing automatically the missing values with the average of their upper and lower neighboring values by column in the dataframe !!!")
print()
for i in range(1,len(dataset.columns)):
  name = dataset.columns[i]
  miss_values_index = dataset.loc[pd.isna(dataset[name]), :].index
  print(name)
  print(miss_values_index)
  print("Showing the missing values with their upper and lower neighboring values by column in the dataframe before and after replacing by average")
  print()
  for j in range(0,len(miss_values_index)):
      
      print("Before ",dataset.iat[miss_values_index[j]-1,i],dataset.iat[miss_values_index[j],i],dataset.iat[miss_values_index[j]+1,i])
      if pd.isnull(dataset.iat[miss_values_index[j]+1,i]) : dataset.iat[miss_values_index[j]+1,i] = dataset.iat[miss_values_index[j]-1,i]
      dataset.iat[miss_values_index[j],i] = (dataset.iat[miss_values_index[j]-1,i]+dataset.iat[miss_values_index[j]+1,i])/2
      print("After ",dataset.iat[miss_values_index[j]-1,i],dataset.iat[miss_values_index[j],i],dataset.iat[miss_values_index[j]+1,i])
      print()

Replacing automatically the missing values with the average of their upper and lower neighboring values by column in the dataframe !!!
Outflow HHP1
Int64Index([27, 30, 51, 55, 62, 64, 80, 100], dtype='int64')
Showing the missing values with their upper and lower neighboring values by column in the dataframe before and after replacing by average
Before  0.0 nan 0.0
After  0.0 0.0 0.0
Before  0.0 nan 0.0
After  0.0 0.0 0.0
Before  0.0 nan 0.0
After  0.0 0.0 0.0
Before  0.0 nan 4.270331085131053
After  0.0 2.1351655425655265 4.270331085131053
Before  0.0 nan 0.0
After  0.0 0.0 0.0
Before  0.0 nan 0.0
After  0.0 0.0 0.0
Before  205.6601600813247 nan 140.81022830168934
After  205.6601600813247 173.23519419150702 140.81022830168934
Before  399.96613186992596 nan 256.4294869743793
After  399.96613186992596 328.19780942215266 256.4294869743793
Inflow  HPP1
Int64Index([96], dtype='int64')
Showing the missing values with their upper and lower neighboring values by column in the dataframe before 

In [58]:
missing_values_table(dataset1)

Your selected dataframe has 6 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [59]:
dataset1.head(40)

,dateRep,Outflow HHP1,Inflow HPP1,Inflow HPP2,Production HPP1,Production HPP2
0,2006/01/01,158.871553,257.0,55.0,2870.0,4350.0
1,2006/01/02,168.014161,253.0,59.0,1960.0,4000.0
2,2006/01/03,113.970973,327.0,67.0,1886.0,3740.0
3,2006/01/04,108.877953,309.0,84.0,1280.0,3240.0
4,2006/01/05,73.086605,262.0,79.0,2400.0,5020.0
5,2006/01/06,135.690533,248.0,84.0,2570.0,3440.0
6,2006/01/07,145.517065,230.0,76.0,2450.0,4430.0
7,2006/01/08,138.368740,186.0,65.0,2290.0,4160.0
8,2006/01/09,128.591676,238.0,62.0,2290.0,4000.0
9,2006/01/10,128.064112,227.0,57.0,2150.0,3640.0


In [ ]:
dataset1.to_csv('energy_float_values.csv', index = False)

In [ ]:
print("Finished with cleaning the missing values !!!")